In [1]:
import gzip
from collections import defaultdict
import math
import numpy as np
import pandas as pd
import string
import random
import string
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as MSE
import json
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = []
f = gzip.open('renttherunway_final_data.json.gz', 'r')
for l in f:
    try:
        x = eval(l)
        
        # Data cleaning: convert height, weight, and rating to integer types
        x['rating'] = int(x['rating'])

        if 'age' in x:
            x['age'] = int(x['age'])
        if 'weight' in x:
            x['weight'] = int(x['weight'][:-3])
        if 'height' in x:
            feet = int(x['height'][0])
            inches = int(x['height'][-3:-1].strip(" "))
            x['height'] = (feet * 12) + inches

        data.append(x)
    except NameError:
        continue
for l in f:
    try:
        x = eval(l)
        data.append(x)
    except NameError:
        continue

In [5]:
df = pd.DataFrame(data)
df.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137.0,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,14,28.0,"April 20, 2016"
1,fit,273551,34b,153475,132.0,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,12,36.0,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,64.0,4,116.0,"December 14, 2015"
3,fit,909926,34c,126335,135.0,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145.0,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,12,27.0,"September 26, 2016"


In [12]:
pt = pd.pivot_table(df, index='fit', columns=['rented for'], values=['rating'])
pt

rating                                              \
rented for      date  everyday formal affair     other     party   
fit                                                                
fit         9.208365  9.144728      9.395493  9.317797  9.280411   
large       8.500535  8.308729      8.701059  8.634266  8.538614   
small       8.197987  7.746820      8.497739  8.458333  8.396081   

                                                          
rented for party: cocktail  vacation   wedding      work  
fit                                                       
fit                    8.0  9.256868  9.362987  9.138252  
large                  NaN  8.403587  8.709906  8.233507  
small                  NaN  8.134146  8.663126  7.950027

In [14]:
pt.index, pt.columns

(Index(['fit', 'large', 'small'], dtype='object', name='fit'),
 MultiIndex([('rating',            'date'),
             ('rating',        'everyday'),
             ('rating',   'formal affair'),
             ('rating',           'other'),
             ('rating',           'party'),
             ('rating', 'party: cocktail'),
             ('rating',        'vacation'),
             ('rating',         'wedding'),
             ('rating',            'work')],
            names=[None, 'rented for']))

In [19]:
pt.loc['fit', ('rating', 'date')]

9.208365019011406

In [22]:
pt.mean(axis=0)

        rented for     
rating  date               8.635629
        everyday           8.400092
        formal affair      8.864764
        other              8.803465
        party              8.738369
        party: cocktail    8.000000
        vacation           8.598201
        wedding            8.912006
        work               8.440595
dtype: float64